In [1]:
using Dates

include("src/ops/Collect.jl");
include("src/ops/Lag.jl");
include("src/ops/Diff.jl");
include("src/ops/Func.jl");
include("src/ops/Aggregate.jl");
include("src/stats/Mean.jl");
include("src/macros.jl");

const f32 = Float32;
const f64 = Float64;
const i32 = Int32;
const i64 = Int64;

In [5]:
function run()
    data = [(DateTime(2020, 1, 1, 0, 0, i), f32(i * 100)) for i in 1:3:40]

    n_features = 11
    out = [f32[] for _ in 1:n_features]

    pipe = @streamops begin
        @broadcast begin
            Transform(x -> x[2]) |> Collect(out[1])                       # X
            Transform(x -> x[2]) |> Lag{f32}(1) |> Collect(out[2])        # X_{t-1}
            Transform(x -> x[2]) |> Lag{f32}(2) |> Collect(out[3])        # X_{t-2}
            Transform(x -> x[2]) |> Diff{f32}(1) |> Collect(out[4])       # X_t - X_{t-1}
            Transform(x -> x[2]) |> Diff{f32}(2) |> Collect(out[5])       # X_t - X_{t-2}
            Transform(x -> x[2]) |> Diff{f32}(3) |> Collect(out[6])       # X_t - X_{t-3}
            Transform(x -> x[2]) |> Mean{f32,f32}(3) |> Collect(out[7])   # mean(X_{t-3}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(6) |> Collect(out[8])   # mean(X_{t-6}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(12) |> Collect(out[9])  # mean(X_{t-12}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(24) |> Collect(out[10]) # mean(X_{t-24}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(48) |> Collect(out[11]) # mean(X_{t-48}, ..., X_t)
        end
        Transform(x -> x[1])
    end

    dates = pipe.(data)
    mat = reduce(hcat, out)
    dates, mat
end

dates, mat = @time run()

display(dates)
display(mat)

14-element Vector{DateTime}:
 2020-01-01T00:00:01
 2020-01-01T00:00:04
 2020-01-01T00:00:07
 2020-01-01T00:00:10
 2020-01-01T00:00:13
 2020-01-01T00:00:16
 2020-01-01T00:00:19
 2020-01-01T00:00:22
 2020-01-01T00:00:25
 2020-01-01T00:00:28
 2020-01-01T00:00:31
 2020-01-01T00:00:34
 2020-01-01T00:00:37
 2020-01-01T00:00:40

14×11 Matrix{Float32}:
  100.0     0.0     0.0  100.0  100.0  …   100.0   100.0   100.0   100.0
  400.0   100.0     0.0  300.0  400.0      250.0   250.0   250.0   250.0
  700.0   400.0   100.0  300.0  600.0      400.0   400.0   400.0   400.0
 1000.0   700.0   400.0  300.0  600.0      550.0   550.0   550.0   550.0
 1300.0  1000.0   700.0  300.0  600.0      700.0   700.0   700.0   700.0
 1600.0  1300.0  1000.0  300.0  600.0  …   850.0   850.0   850.0   850.0
 1900.0  1600.0  1300.0  300.0  600.0     1150.0  1000.0  1000.0  1000.0
 2200.0  1900.0  1600.0  300.0  600.0     1450.0  1150.0  1150.0  1150.0
 2500.0  2200.0  1900.0  300.0  600.0     1750.0  1300.0  1300.0  1300.0
 2800.0  2500.0  2200.0  300.0  600.0     2050.0  1450.0  1450.0  1450.0
 3100.0  2800.0  2500.0  300.0  600.0  …  2350.0  1600.0  1600.0  1600.0
 3400.0  3100.0  2800.0  300.0  600.0     2650.0  1750.0  1750.0  1750.0
 3700.0  3400.0  3100.0  300.0  600.0     2950.0  2050.0  1900.0  1900.0
 4000.0  3700.0  3400.0  300

  0.000011 seconds (62 allocations: 5.891 KiB)


In [4]:
function run()
    data = [(DateTime(2020, 1, 1, 0, 0, i), f32(i * 100)) for i in 1:3:40]

    n_features = 11
    out = [f32[] for _ in 1:n_features]

    pipe = @streamops begin
        @broadcast_collect begin
            Transform(x -> x[2])                       # X
            Transform(x -> x[2]) |> Lag{f32}(1)        # X_{t-1}
            Transform(x -> x[2]) |> Lag{f32}(2)        # X_{t-2}
            Transform(x -> x[2]) |> Diff{f32}(1)       # X_t - X_{t-1}
            Transform(x -> x[2]) |> Diff{f32}(2)       # X_t - X_{t-2}
            Transform(x -> x[2]) |> Diff{f32}(3)       # X_t - X_{t-3}
            Transform(x -> x[2]) |> Mean{f32,f32}(3)   # mean(X_{t-3}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(6)   # mean(X_{t-6}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(12)  # mean(X_{t-12}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(24)  # mean(X_{t-24}, ..., X_t)
            Transform(x -> x[2]) |> Mean{f32,f32}(48)  # mean(X_{t-48}, ..., X_t)
        end
    end

    pipe.(data)
end

@time run()

  0.000009 seconds (39 allocations: 3.312 KiB)


14-element Vector{NTuple{11, Float32}}:
 (100.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0)
 (400.0, 100.0, 0.0, 300.0, 400.0, 400.0, 250.0, 250.0, 250.0, 250.0, 250.0)
 (700.0, 400.0, 100.0, 300.0, 600.0, 700.0, 400.0, 400.0, 400.0, 400.0, 400.0)
 (1000.0, 700.0, 400.0, 300.0, 600.0, 900.0, 700.0, 550.0, 550.0, 550.0, 550.0)
 (1300.0, 1000.0, 700.0, 300.0, 600.0, 900.0, 1000.0, 700.0, 700.0, 700.0, 700.0)
 (1600.0, 1300.0, 1000.0, 300.0, 600.0, 900.0, 1300.0, 850.0, 850.0, 850.0, 850.0)
 (1900.0, 1600.0, 1300.0, 300.0, 600.0, 900.0, 1600.0, 1150.0, 1000.0, 1000.0, 1000.0)
 (2200.0, 1900.0, 1600.0, 300.0, 600.0, 900.0, 1900.0, 1450.0, 1150.0, 1150.0, 1150.0)
 (2500.0, 2200.0, 1900.0, 300.0, 600.0, 900.0, 2200.0, 1750.0, 1300.0, 1300.0, 1300.0)
 (2800.0, 2500.0, 2200.0, 300.0, 600.0, 900.0, 2500.0, 2050.0, 1450.0, 1450.0, 1450.0)
 (3100.0, 2800.0, 2500.0, 300.0, 600.0, 900.0, 2800.0, 2350.0, 1600.0, 1600.0, 1600.0)
 (3400.0, 3100.0, 2800.0, 300.0, 600.0, 900.0, 31